<a href="https://colab.research.google.com/github/Sabrualvarez/Minox/blob/main/IA_BUENO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 📦 Instala librerías necesarias (por si no están)
!pip install pandas openpyxl


In [24]:
# 📚 1. Importaciones necesarias
import pandas as pd

In [25]:
import pandas as pd

# We can use the read_excel method to read data from an Excel file
df = pd.read_excel('/content/todo_junto.xlsx')

# Let's check out the first 5 rows from our excel file
df.head()

,Serie,Modelo,Aumento,Control por app,Abertura,Recorrido de ajuste (MOA),Diámetro de lente (mm),Localización GPS,Ajuste por clic (MOA),Tamaño del punto rojo (MOA),...,Rosca trípode,Temperatura de funcionamiento,Servicio MINOX Comfort,Resistencia al agua,Protección salpicaduras,Contenido del paquete,Dimensiones,Peso (g),Nº material,Precio
0,Cámara trampeo,DTC 1200,NaN,"Si (gratis, Android e iOS)",NaN,NaN,NaN,Si,NaN,NaN,...,Si,NaN,NaN,NaN,Si,"Cámara, SIM, antena 4G, microSD 16 GB, correa",NaN,460.0,80406283.0,NaN
1,Cámara trampeo,DTC 550 WiFi,NaN,"Si (gratis, Android e iOS)",NaN,NaN,NaN,No,NaN,NaN,...,Si,NaN,NaN,NaN,Si,"Correa, cable USB, soporte pared",NaN,290.0,80407270.0,188.0
2,Cámara trampeo,DTC 550,NaN,No,NaN,NaN,NaN,No,NaN,NaN,...,Si,NaN,NaN,NaN,Si,"Cámara, correa, soporte, cable USB, instrucciones",NaN,290.0,80405434.0,NaN
3,Cámara trampeo,DTC 460,NaN,No,NaN,NaN,NaN,No,NaN,NaN,...,Si,NaN,NaN,NaN,Si,"Cámara, correa, soporte, cable USB, guía rápida",NaN,415.0,80406281.0,159.0
4,Cámara trampeo,DTC 395,NaN,No,NaN,NaN,NaN,No,NaN,NaN,...,Si,NaN,NaN,NaN,Si,"Correa, cable USB, soporte pared",NaN,280.0,80405444.0,135.0


In [26]:
# Convertir columnas a minúsculas para evitar errores
df.columns = [col.strip().lower() for col in df.columns]

In [27]:
# Buscar modelos por serie y mostrar tabla con serie + modelo
def buscar_modelos_por_serie(serie):
    serie = serie.strip().lower()

    # Filtrar registros que contengan el nombre parcial de la serie
    resultados = df[df['serie'].str.lower().str.contains(serie, na=False)]

    if resultados.empty:
        return f"No se encontraron modelos que pertenezcan a una serie que contenga: '{serie}'."

    # Seleccionar solo columnas deseadas y eliminar duplicados
    tabla = resultados[['serie', 'modelo']].drop_duplicates()

    # Devolver el DataFrame directamente para que Colab lo muestre en formato tabla
    return tabla.reset_index(drop=True)


In [28]:
buscar_modelos_por_serie("RS-4")

,serie,modelo
0,RS-4,1-6x24 (S)
1,RS-4,1.6-10x42 (S)
2,RS-4,2.2-14x50 (S)
3,RS-4,3-18x56 (S)
4,RS-4,1-4x24
5,RS-4,2.5-10x50
6,RS-4,2.5-10x42
7,RS-4,3-12x56


In [34]:
# Características de uno o varios modelos con formato de moneda en el precio
def caracteristicas_modelo(nombre_modelo):
    nombre_modelo = nombre_modelo.strip().lower()

    # Filtrar modelos que contengan ese texto
    coincidencias = df[df['modelo'].str.lower().str.contains(nombre_modelo, na=False)]

    if coincidencias.empty:
        return f"No se encontraron modelos que contengan '{nombre_modelo}'."

    columnas_deseadas = ['modelo', 'serie', 'aumento', 'dimensiones', 'peso (g)', 'nº material', 'precio']
    columnas_existentes = [col for col in columnas_deseadas if col in df.columns]
    tabla = coincidencias[columnas_existentes].copy()

    # 👉 Formatear la columna "precio" si existe
    if 'precio' in tabla.columns:
        tabla['precio'] = tabla['precio'].apply(lambda x: f"{x} €" if pd.notnull(x) else x)

    return tabla.reset_index(drop=True)


In [35]:
caracteristicas_modelo("DTC 550 WiFi")

,modelo,serie,aumento,dimensiones,peso (g),nº material,precio
0,DTC 550 WiFi,Cámara trampeo,NaN,NaN,290.0,80407270.0,188.0 €


In [78]:
import re

def precio_de_modelo(filtro):
    filtro = filtro.strip().lower()

    # Si se solicita ver todos los productos
    if filtro == "" or filtro == "todos":
        resultados = df[['modelo', 'precio']].dropna().copy()

    # Si es un rango tipo "entre 100 y 200"
    elif ("entre" in filtro and "y" in filtro) or ("entre" in filtro and "-" in filtro):
        numeros = re.findall(r"\d+(?:[.,]\d+)?", filtro)
        if len(numeros) == 2:
            min_val = float(numeros[0].replace(",", "."))
            max_val = float(numeros[1].replace(",", "."))
            resultados = df[(df['precio'] >= min_val) & (df['precio'] <= max_val)][['modelo', 'precio']].copy()
        else:
            return "No entendí bien el rango. Usa por ejemplo: 'entre 100 y 300'"

    # Si es tipo "mayor de 300"
    elif "mayor de" in filtro:
        numero = re.search(r"\d+(?:[.,]\d+)?", filtro)
        if numero:
            val = float(numero.group().replace(",", "."))
            resultados = df[df['precio'] > val][['modelo', 'precio']].copy()
        else:
            return "Indica un número válido después de 'mayor de'."

    # Si es tipo "menor de 300"
    elif "menor de" in filtro:
        numero = re.search(r"\d+(?:[.,]\d+)?", filtro)
        if numero:
            val = float(numero.group().replace(",", "."))
            resultados = df[df['precio'] < val][['modelo', 'precio']].copy()
        else:
            return "Indica un número válido después de 'menor de'."

    # Si es búsqueda por nombre parcial del modelo
    else:
        resultados = df[df['modelo'].str.lower().str.contains(filtro, na=False)][['modelo', 'precio']].copy()

    # Si no se encuentra nada
    if resultados.empty:
        print ("Prueba con:")
        print("- entre 100 y 200")
        print("- mayor de 300")
        print("- menor de 250")
        print("- DTC")
        print("- Todos")

    # Ordenar y dar formato
    resultados = resultados.drop_duplicates().sort_values(by='precio')
    resultados['precio'] = resultados['precio'].apply(lambda x: f"{x:,.2f} €" if pd.notnull(x) else x)

    return resultados.reset_index(drop=True)


In [79]:
precio_de_modelo("mayor")

Prueba con:
- entre 100 y 200
- mayor de 300
- menor de 250
- DTC
- Todos


,modelo,precio


In [ ]:
def chatbot():
    print("🤖 ¡Hola! Soy tu asistente de Minox.")
    print("Puedes preguntarme cosas como:")
    print("Modelo de la serie"(ej: "modelo de la serie RS-4"))
    print("- Caracteristicas del modelo (ej: 'Modelo DTC 1200')")
    print("- Precio de los productos (ej: 'Precio de DTC 1200')")
    print("- Dimensiones de los produstos (ej: 'Dimensiones de DTC 1200)")
    print("Escribe 'salir' para terminar.\n")

    while True:
        pregunta = input("🟢 Tú: ").lower()

        if "salir" in pregunta:
            print("👋 ¡Hasta luego!")
            break

        elif "cliente" in pregunta or "compra" in pregunta:
            nombre = input("🔹 ¿Nombre del cliente?: ")
            buscar_por_cliente(nombre)

        elif "producto" in pregunta or "venta de" in pregunta or "ventas de" in pregunta:
            nombre = input("🔹 ¿Nombre del producto?: ")
            buscar_por_producto(nombre)

        elif "fecha" in pregunta or "día" in pregunta or "ventas del" in pregunta:
            fecha = input("🔹 ¿Qué fecha? (formato YYYY-MM-DD): ")
            buscar_por_fecha(fecha)

        else:
            print("❓ No entendí la pregunta. Intenta usar palabras como 'cliente', 'producto' o 'fecha'.")


In [ ]:
# 📦 Instala librerías necesarias (por si no están)
!pip install pandas openpyxl

# 📚 1. Importaciones necesarias
import pandas as pd

# 📁 2. Cargar el archivo de Excel desde Google Colab
from google.colab import files
uploaded = files.upload()

# ⬇️ 3. Leer el Excel cargado
excel_filename = list(uploaded.keys())[0]
df = pd.read_excel(excel_filename)

# 🧼 4. Filtramos solo las columnas de interés
columnas_interes = ['precio', 'número de material', 'peso', 'dimensiones', 'modelo', 'aumentos']
df = df[[col for col in columnas_interes if col in df.columns]]

# 👁️ 5. Mostrar un resumen para comprobar que todo está bien
print("Columnas disponibles:")
print(df.columns)
print("\nPrimeras filas de los datos:")
print(df.head())

# 🤖 6. Mini sistema de consulta
def responder_pregunta(pregunta):
    pregunta = pregunta.lower()
    respuesta = ""

    if "precio" in pregunta:
        respuesta = df[['modelo', 'precio']].dropna().to_string(index=False)
    elif "material" in pregunta or "número" in pregunta:
        respuesta = df[['modelo', 'número de material']].dropna().to_string(index=False)
    elif "peso" in pregunta:
        respuesta = df[['modelo', 'peso']].dropna().to_string(index=False)
    elif "dimensiones" in pregunta:
        respuesta = df[['modelo', 'dimensiones']].dropna().to_string(index=False)
    elif "aumento" in pregunta:
        respuesta = df[['modelo', 'aumentos']].dropna().to_string(index=False)
    elif "modelo" in pregunta:
        respuesta = df['modelo'].dropna().to_string(index=False)
    else:
        respuesta = "Lo siento, no he entendido tu pregunta. Intenta usar palabras clave como 'precio', 'peso', 'dimensiones', etc."

    return respuesta

# 🧠 7. Interacción con el usuario
while True:
    pregunta = input("¿Qué deseas conocer sobre los productos Minox? (Escribe 'salir' para terminar)\n")
    if pregunta.lower() == 'salir':
        print("Gracias por usar el sistema. ¡Hasta luego!")
        break
    print("\nRespuesta:")
    print(responder_pregunta(pregunta))
    print("-" * 50)
